<a href="https://colab.research.google.com/github/jaunty0216/ForSophomoreCourse/blob/main/%E7%AC%AC%E5%85%AB%E9%80%B1_%E4%BD%A0%E5%80%91%E8%AA%AA%E5%95%A5%E5%91%A2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 目標

今天要把~~網路上的蟲給爬了~~

還把說的話的**內容給分析**了

話說我意外學會了**建立子目錄**的方式(

# 進度規劃


1.   生成留言內容(謝謝chatGPT)
2.   將生成的csv檔案上傳
4.   開始分析留言內容
5.   畫出字雲圖
7.   資料結果分析



## 上傳生成的內容

In [27]:
from google.colab import files
uploaded = files.upload()

for filename,_ in uploaded.items():
    filepath = f"/content/{filename}"
    print(f"Filepath: {filepath}")
    # 在這裡您可以使用 filepath 來處理檔案

Saving scifi_novel_reviews_varied_utf8.csv to scifi_novel_reviews_varied_utf8.csv
Filepath: /content/scifi_novel_reviews_varied_utf8.csv


## 將資料放到放到google sheet

In [28]:
import pandas as pd
OriginFile = pd.read_csv(filepath)
Origin_df = pd.DataFrame(OriginFile)
Origin_df.head()

#我先檢查

,user_id,rating,comment,translation
0,user_001,10.0,Great book overall. It tackled complex ideas i...,總的來說是一本很棒的書。它以一種易於理解的方式處理複雜的想法。我很快就會重讀這篇文章。
1,user_002,10.0,"An absolute gem! From the plot to the pacing, ...",絕對的寶石！從情節到節奏，一切都很順利。我還在想它觸及的一些主題。
2,user_003,9.7,"An absolute gem! From the plot to the pacing, ...",絕對的寶石！從情節到節奏，一切都很順利。我還在想它觸及的一些主題。
3,user_004,8.7,"A slow start, but once it picked up, I couldn'...",開始很慢，但一旦開始，我就無法停止閱讀。主角的旅程很鼓舞人心。喜歡這種寫作風格。
4,user_005,8.9,I didn? expect to enjoy it this much. The wor...,我沒想到會這麼喜歡它。世界構建是一流的。一些對話感覺很僵硬，但情節彌補了這一點。


In [29]:
#再來萃取
ExtractedContent = Origin_df["translation"]
#ExtractedContent.head()
import pandas as pd
ExtractedContent_df = pd.DataFrame(ExtractedContent)
ExtractedContent_df.head()

,translation
0,總的來說是一本很棒的書。它以一種易於理解的方式處理複雜的想法。我很快就會重讀這篇文章。
1,絕對的寶石！從情節到節奏，一切都很順利。我還在想它觸及的一些主題。
2,絕對的寶石！從情節到節奏，一切都很順利。我還在想它觸及的一些主題。
3,開始很慢，但一旦開始，我就無法停止閱讀。主角的旅程很鼓舞人心。喜歡這種寫作風格。
4,我沒想到會這麼喜歡它。世界構建是一流的。一些對話感覺很僵硬，但情節彌補了這一點。


In [30]:
#把資料放到google sheet 囉
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)
SpreadSheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/16LGXgFCCVwqVU48-JoQzY6SbtIn3Vp6U-_kfjsg-Q1E/edit?usp=sharing")

from gspread_dataframe import set_with_dataframe
Record = SpreadSheet.get_worksheet(0)
set_with_dataframe(Record,ExtractedContent_df)

## 分析留言

In [31]:
# 引入所需套件
!pip install jieba
!pip install snownlp

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 16.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for snownlp: filename=snownlp-0.12.3-py3-none-any.whl size=37760946 sha256=ad92c04bd99c8e0f7b9374f1806e5925e708393d17d26c3c80c4028232f19608
  Stored in directory: /root/.cache/pip/wheels/4a/fc/04/d1937c02b2a445b34754da55f71612a3df648a38d711bd17eb
Successfully built snownlp


In [32]:
rows = Record.get_all_records()
import pandas as pd
rows_df = pd.DataFrame(rows)

In [33]:
import jieba
from snownlp import SnowNLP
import pandas as pd
from collections import Counter
import re
import plotly.express as px

In [34]:
for _, content in ExtractedContent_df.iterrows():
    word = content['translation']
    jieba.add_word(word, freq=10000)
    jieba.suggest_freq(word, tune=True)

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.291 seconds.
DEBUG:jieba:Loading model cost 1.291 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


In [35]:
# 只保留中文字
def extract_chinese(text):
    return ''.join(re.findall(r'[\u4e00-\u9fa5]', text))

In [36]:
ExtractedContent_df['Chinese_Content'] = ExtractedContent_df['translation'].apply(extract_chinese)

# 使用 Jieba 斷詞
ExtractedContent_df['Tokenized'] = ExtractedContent_df['Chinese_Content'].apply(lambda x: list(jieba.cut(x, HMM=True)))

# 展平成所有詞語的列表
all_words = [word for tokens in ExtractedContent_df['Tokenized'] for word in tokens if len(word) > 1]

In [37]:
ExtractedContent_df['Chinese_Content']

,Chinese_Content
0,總的來說是一本很棒的書它以一種易於理解的方式處理複雜的想法我很快就會重讀這篇文章
1,絕對的寶石從情節到節奏一切都很順利我還在想它觸及的一些主題
2,絕對的寶石從情節到節奏一切都很順利我還在想它觸及的一些主題
3,開始很慢但一旦開始我就無法停止閱讀主角的旅程很鼓舞人心喜歡這種寫作風格
4,我沒想到會這麼喜歡它世界構建是一流的一些對話感覺很僵硬但情節彌補了這一點
...,...
195,絕對的寶石從情節到節奏一切都很順利我還在想它觸及的一些主題
196,精彩的故事講述和強烈的角色發展它挑戰了你對人性的看法科幻迷必讀
197,這個故事身臨其境節奏合理我特別喜歡角色面臨的道德困境這是一本陪伴你的書
198,我完全被第二章迷住了作者的想像力是天馬行空的絕對推薦給我的朋友


In [38]:
ExtractedContent_df['Tokenized']

,Tokenized
0,"[總, 的, 來, 說, 是, 一本, 很棒, 的, 書, 它, 以, 一種, 易, 於, ..."
1,"[絕對, 的, 寶石, 從, 情節, 到, 節奏, 一切, 都, 很, 順利, 我, 還在,..."
2,"[絕對, 的, 寶石, 從, 情節, 到, 節奏, 一切, 都, 很, 順利, 我, 還在,..."
3,"[開始, 很慢, 但, 一旦, 開始, 我, 就, 無法, 停止, 閱讀, 主角, 的, 旅..."
4,"[我, 沒, 想到, 會, 這麼, 喜歡, 它, 世界, 構建, 是, 一流, 的, 一些,..."
...,...
195,"[絕對, 的, 寶石, 從, 情節, 到, 節奏, 一切, 都, 很, 順利, 我, 還在,..."
196,"[精彩, 的, 故事, 講述, 和, 強烈, 的, 角色, 發展, 它, 挑戰, 了, 你,..."
197,"[這個, 故事, 身臨, 其境, 節奏, 合理, 我, 特別, 喜歡, 角色, 面臨, 的,..."
198,"[我, 完全, 被, 第二章, 迷住, 了, 作者, 的, 想像力, 是, 天馬, 行空, ..."


In [39]:
from collections import Counter
word_counts = Counter(all_words)
word_counts

Counter({'一本': 39,
         '很棒': 37,
         '一種': 15,
         '理解': 15,
         '方式': 15,
         '處理': 15,
         '雜的': 15,
         '想法': 15,
         '很快': 15,
         '就會': 15,
         '重讀': 15,
         '篇文章': 15,
         '絕對': 45,
         '寶石': 22,
         '情節': 22,
         '節奏': 46,
         '一切': 22,
         '順利': 22,
         '還在': 22,
         '觸及': 22,
         '一些': 63,
         '主題': 22,
         '開始': 36,
         '很慢': 18,
         '一旦': 18,
         '無法': 18,
         '停止': 18,
         '閱讀': 18,
         '主角': 18,
         '旅程': 18,
         '鼓舞人心': 18,
         '喜歡': 81,
         '這種': 18,
         '作風格': 18,
         '想到': 23,
         '這麼': 23,
         '世界': 23,
         '構建': 23,
         '一流': 23,
         '對話': 23,
         '感覺': 39,
         '僵硬': 23,
         '但情': 23,
         '節彌補': 23,
         '這一點': 23,
         '這個': 24,
         '故事': 59,
         '身臨': 24,
         '其境': 24,
         '合理': 24,
         '特別': 24,
         '角色': 43,
      

In [46]:
# 排除停用詞（這裡提供一個簡單的停用詞列表，可根據需要擴展）
stop_words = set(['所以', '好', '因為', '大家', '的','但情', '是', '了', '我', '也', '在', '和', '就', '不', '有', '他', '她', '你', '我們', '這個','節彌補'])
filtered_word_counts = {word: count for word, count in word_counts.items() if word not in stop_words}

原句：word: count for word, count in word_counts.items() if word not in stop_words


*   word: count：分別**對應到key-valuse**
*   for word, count in word_counts.items()：word_counts.items()會產出**兩個參數**
*   if word not in stop_words：**篩選**條件

In [41]:
# 將詞頻轉為 DataFrame 並排序
word_freq_df = pd.DataFrame(filtered_word_counts.items(), columns=['Word', 'Frequency'])
word_freq_df = word_freq_df.sort_values(by='Frequency', ascending=False).reset_index(drop=True)

## 畫出字雲圖

In [42]:
fig = px.bar(word_freq_df.head(30), x='Word', y='Frequency',
             title='關鍵字出現次數統計（前20名）',
             labels={'Word': '關鍵字', 'Frequency': '出現次數'},
             text='Frequency')

fig.update_layout(
    xaxis_title="關鍵字",
    yaxis_title="出現次數",
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True),
    template='plotly_white'
)

fig.show()

In [43]:
!apt-get update -qq
!apt-get install -y fonts-wqy-zenhei

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  fonts-wqy-zenhei
0 upgraded, 1 newly installed, 0 to remove and 89 not upgraded.
Need to get 7,472 kB of archives.
After this operation, 16.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 fonts-wqy-zenhei all 0.9.45-8 [7,472 kB]
Fetched 7,472 kB in 1s (8,317 kB/s)
Selecting previously unselected package fonts-wqy-zenhei.
(Reading database ... 126102 files and directories currently installed.)
Preparing to unpack .../fonts-wqy-zenhei_0.9.45-8_all.deb ...
Unpacking fonts-wqy-zenhei (0.9.45-8) ...
Setting up fonts-wqy-zenhei (0.9.45-8) ...
Processing triggers for fontconfig (2.13.1-4.2ubuntu5) ...


In [53]:
from wordcloud import WordCloud

# 取前 300 個最常出現的詞
top_words = word_freq_df.head(300)

# 使用 WQY Zenhei 字體(建立wordcloud物件)
wordcloud = WordCloud(
    width=800,
    height=400,
    background_color='white',
    font_path='/usr/share/fonts/truetype/wqy/wqy-zenhei.ttc'
)

# 根據詞頻生成文字雲(使用wordcloud物件執行生成作業)
#generate_from_frequencies要接收的型態是dict
wordcloud.generate_from_frequencies(dict(zip(top_words['Word'], top_words['Frequency'])))

# 將文字雲轉換為圖像(生成numpy數據)
wordcloud_img = wordcloud.to_array()

# 使用 Plotly 顯示文字雲
fig = px.imshow(wordcloud_img)
fig.update_layout(
    title="文字雲",
    xaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    yaxis=dict(showgrid=False, showticklabels=False, zeroline=False),
    coloraxis_showscale=False
)
fig.show()

## 資料結果分析

In [45]:
from google.colab import userdata
apikey = userdata.get('GOOGLE_API_KEY')
import google.generativeai as genai
genai.configure(api_key = apikey)
model = genai.GenerativeModel("gemini-1.5-flash")

prompt = f"請摘要出這些評論的重點，並補上重點中的情緒表現，把最多人呈現的情緒及其相關的內容整併在一起，說明這些發文的風向：{ExtractedContent_df.to_string()}"

response = model.generate_content(prompt)
print(response.text)

這些評論主要表達了**強烈正面**的情緒，對這本書給予高度評價。  最常見的情緒是**喜愛（positive affection）**和**驚嘆（awe/amazement）**。

以下是根據評論中出現頻率，將最多人呈現的情緒及其相關內容整併在一起的摘要：

**1. 強烈推薦及喜愛：**  許多評論使用“很棒”、“絕對的寶石”等詞彙，表達對本書的強烈喜愛。讀者表示他們意想不到地喜歡這本書，並會重讀或推薦給朋友。這部分情緒展現了讀者對劇情、節奏、世界構建和整體閱讀體驗的滿意。 （例：評論 0, 1, 2, 7, 15, 22, 28, 32, 42, 46, 49, 51, 53, 54, 60, 69, 71, 73, 77, 83, 84, 86, 87, 98, 100, 104, 106, 112, 129, 147, 151, 160, 162, 164, 172, 186, 188, 195, 198）


**2.  劇情和節奏的讚賞：**  許多評論讚揚了本書的劇情、節奏和世界構建（world-building）。  讀者覺得劇情引人入勝、節奏合理，甚至描述為“身臨其境”。有些讀者還特別點出了角色發展、道德困境等方面吸引人的地方。（例：評論 1, 2, 6, 8, 23, 26, 27, 50, 58, 79, 101, 118, 133, 158, 171, 175, 178, 180, 189, 193, 197, 199）


**3.  深刻的思考和創意：** 部分評論提到這本書引發了他們對社會、AI 和人性的深刻思考，並讚揚了作者的想像力以及科學與情感的完美結合。  這些評論展現了讀者對作品深度和創意的驚嘆。（例：評論 9, 12, 13, 14, 35, 37, 38, 43, 45, 47, 48, 52, 55, 57, 61, 63, 64, 65, 66, 68, 72, 78, 81, 82, 88, 90, 91, 92, 93, 96, 97, 108, 109, 110, 111, 113, 114, 115, 116, 122, 123, 124, 125, 130, 131, 132, 134, 136, 139, 143, 146, 149, 150, 153, 154, 155, 159, 163, 165, 1

# 事後筆記

**心得**

在這一次的實作裡，我學到了怎麼用圖像化的方式更好的分析文本內容

對我來說，這還能夠用做藝術用途

**產生文字雲的邏輯順序**


1.   引入資料
2.   讀取資料
3.   將資料轉為dataframe
4.   使用jieba斷字
5.   過濾內容
6.   使用collection製作 單字:頻率的字典
7.   使用wordcloud和plotly.express製圖

